## Landmark Detection

This code uses dlib face detector(http://dlib.net/) to determine (x,y) coordinates of 81 landmarks. The input is video files, and the output is a matrix for each frame. These matrices serve as inputs for MST Map generation.  

In [ ]:
import argparse
import imutils
import time
import dlib
import cv2
import os
from skimage import io
import numpy as np
import glob
import sys
import scipy.io
import re
import time
from matplotlib import pyplot as plt 
import pylab 
pylab.rcParams['figure.figsize'] = (10.0, 8.0) 

In [ ]:
#initialize paths and variables
#os.chdir(Video Path) Change to video path if not videos saved in same folder as file 
path = os.getcwd()

videos = glob.glob(os.path.join(path,"subject*/vid.avi"))

In [ ]:
def getFrames(vid_path, save_path, subject_number):
    frame_list = [];
    vidObj = cv2.VideoCapture(vid_path)
    if vidObj.isOpened():
        current_frame = 0
        while True:
            ret, frame = vidObj.read()
            if ret:
                name = f"{save_path}/frame_{current_frame}.jpg" 
                cv2.imwrite(name,frame)
                current_frame +=1
                frame_list.append(name)
            else:
                break
        vidObj.release()
    cv2.destroyAllWindows()
    print(f"[Info] Frames successful for subject {subject_number}")
    return frame_list

In [ ]:
def make_detector_matrix(image_filename):

    detector = dlib.get_frontal_face_detector() #load in detector
    image = cv2.imread(image_filename)
    #image = imutils.resize(image, width=600) #changing to test if this is causing the issue
    rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    predictor_path = os.path.join(path,"/Landmark_Detection/shape_predictor_81_face_landmarks.dat")
    
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor(predictor_path)
    dets = detector(rgb,0)

    for k,d in enumerate(dets):
        shape = predictor(rgb,d)
        landmarks = np.array([[p.x,p.y] for p in shape.parts()]) #!!!!! This is the landmarks file

    reshaped = np.reshape(landmarks,(162,1))

    new_matrix = np.zeros(162)
    for i in range(0,162):
        new_matrix[i] = int(reshaped[i][0])
    new_matrix_dict = {'landmarks': new_matrix}
    return new_matrix_dict

In [ ]:
for i in videos:
    start_time = time.time()
    subject_number = i[-10:-8]
    try:
        subject_number = subject_number.replace('t','')
    except:
        pass
    print(i)
    directory = f"/Landmarks/{subject_number}"
    make_path = os.path.join(path,directory)
    make_path2 = os.path.join(make_path,"arrays")
    #os.mkdir(make_path)
    os.mkdir(make_path2)
    
    frame_names = getFrames(i,make_path,subject_number)
    #frame_sorted = sorted(glob.glob(f"Code/Landmarks/{subject_number}/*.jpg"), key=lambda x: int(re.findall(r'\d+', x)[0]))
    #note: frame_names is already sorted
    
    current_frame = -1 #initializing for array loop (so it will start at 0)
    for j in frame_names:
        current_frame+=1
        try:
            new_matrix_dict = make_detector_matrix(j)
            scipy.io.savemat(f'{make_path2}/frame_{current_frame}.mat',new_matrix_dict)
        except:
            scipy.io.savemat(f'{make_path2}/frame_{current_frame}.mat',new_matrix_dict)
            print(f"Error detecting landmarks for frame {current_frame}. Using previous frame")
    print(f"Subject {subject_number} took {time.time()-start_time} long")
    